In [1]:
import pyodbc
import clipboard
import urllib
from sqlalchemy import create_engine
import pandas as pd
import math
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
xl_path = 'data/AKC_Source To Target Mapping_v1.1.xlsx'
server = 'dms-kbi.database.windows.net'
database = 'dms-kpi'
username = 'kbiadmin'
password = 'aQw$dfXzc@31'
driver= '{ODBC Driver 17 for SQL Server}'

conn= pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor =  conn.cursor()
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
sql = "SELECT * FROM FND.V_KPI_KPIASSIGNMENT"
data = pd.read_sql(sql,conn)

xls = pd.ExcelFile(xl_path)
xl2 = pd.ExcelFile(xl_path)

C:\Users\vinh2_ri4orrg\AppData\Local\Temp\ipykernel_17472\4204459091.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql,conn)


In [7]:
fact_name = 'F_SALES_INVOICE_DAILY'
schema = 'DWH'

In [3]:
#only get the necessary columns in specific order
col_list = ['Object Name','Attribute Name','Datatype','Object Name.1','Attribute Name.1']
sheet_list = []

sheets = xls.book.worksheets
for sheet in sheets:
    if sheet.sheet_state == 'visible':
        sheet_list.append(sheet.title)
        print(sheet.title)

List Table
1. STTM - DIM
4. STTM - MAP
2. STTM - FACT
3. STTM - V_FACT


In [4]:
sheet2 = pd.read_excel(open(xl_path, 'rb'),sheet_name=sheet_list[3],skiprows=2)
sheet2

,Schema/Module,Object Name,Attribute Name,Datatype,Source Name,Object Name.1,Attribute Name.1
0,FACT,F_DETAIL_VISIT_PLAN,W_INSERT_DT,datetime,NaN,NaN,NaN
1,FACT,F_DETAIL_VISIT_PLAN,W_UPDATE_DT,datetime,NaN,NaN,NaN
2,FACT,F_DETAIL_VISIT_PLAN,W_INTEGRATION_ID,nvarchar,eMobizSTG,STG.EMB_DETAILVISITPLAN,"1. SELECT B.DISTRIBUTOR_KEY,C.CUSTOMER_KEY,D.S..."
3,FACT,F_DETAIL_VISIT_PLAN,W_SOURCE,nvarchar,eMobizSTG,STG.EMB_DETAILVISITPLAN,DEFAULT = 'EMB_DETAILVISITPLAN'
4,FACT,F_DETAIL_VISIT_PLAN,DISTRIBUTOR_KEY,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,B.DISTRIBUTOR_KEY
5,FACT,F_DETAIL_VISIT_PLAN,CUSTOMER_KEY,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,C.CUSTOMER_KEY
6,FACT,F_DETAIL_VISIT_PLAN,SALES_FORCE_KEY,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,D.SALES_FORCE_KEY
7,FACT,F_DETAIL_VISIT_PLAN,EMPLOYEE_KEY,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,E.EMPLOYEE_KEY
8,FACT,F_DETAIL_VISIT_PLAN,DATE_KEY,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,VISITDATE
9,FACT,F_DETAIL_VISIT_PLAN,YEAR,int,eMobizSTG,STG.EMB_DETAILVISITPLAN,YEAR(VISITDATE)


In [5]:
col_order_map = {
    'W_INTEGRATION_ID':0,
    'W_SOURCE':10,
    'W_INSERT_DT':11,
    'W_UPDATE_DT':12,

}

In [9]:
def preprocessing(v):
    value = v.upper()
    value = value.strip()
    value = value.replace('\n','')
    value = value.replace('\r','')
    return value
def handleLogic(formula,alias):
    if (isinstance(formula,float)):
        if(math.isnan(formula)):
            return ','+alias
    else:
        src = formula.upper()
        dst = alias.upper()
        #handle case default
        if ('DEFAULT' in src):
            src = src.replace('DEFAULT','')
            src = src.replace('=','')
            src = src.strip()
        #add cte to w_integration_id
        if (dst == 'W_INTEGRATION_ID'):
            src = src.replace('STG.','FND.')
            return 'CREATE OR ALTER VIEW '+ schema+'.V_'+fact_name+' AS\n' \
                   +'WITH CTE AS (\n' + src+' AS '+dst +'\n)\nSELECT'

        if ( '.' in src):
            src = src[0:src.find('.')-1] +'A'+ src[src.find('.'):]
        if('IF ' in src):
            src = src.replace('IF','IIF(')
            src = src.replace('THEN',',')
            src = src.replace('ELSE',',')
            return ','+src+') AS '+dst
        if ('_KEY' in dst and dst != 'DATE_KEY'):
            src = 'ISNULL('+src+',-1)'
        return ','+src+' AS '+dst
def createOrder(v):
    value = preprocessing(v)
    if value in col_order_map:
        return col_order_map[value]
    else: return 1

df = sheet2[sheet2['Object Name'] == fact_name].copy(deep=True)
df['test_function']=df.apply(lambda x:handleLogic(x['Attribute Name.1'],x['Attribute Name']),axis=1)
df['order'] = df.apply(lambda x:createOrder(x['Attribute Name']),axis=1)

df.sort_values('order',inplace=True,ascending=True)
df['test_function'].to_csv('dump_to_csv.csv',index=False)
df['test_function'].to_clipboard(index = False)
one_string = '\n'.join(df['test_function'].tolist())
one_string = one_string +'\nFROM CTE A'
clipboard.copy(one_string)
print(one_string)

CREATE OR ALTER VIEW DWH.V_F_SALES_INVOICE_DAILY AS
WITH CTE AS (
1. LẤY F_SALES_INVOICE_DETAIL - F_SALES_RETURN_DETAIL
GROUP BY THEO DISTRIBUTOR_KEY, CUSTOMER_KEY, EMPLOYEE_KEY, SALES_FORCE_KEY, PRODUCT_KEY, DATE_KEY, ITEM_TYPE
2. DISTRIBUTOR_KEY~CUSTOMER_KEY~EMPLOYEE_KEY~SALES_FORCE_KEY~PRODUCT_KEY~DATE_KEY~ITEM_TYPE AS W_INTEGRATION_ID
)
SELECT
,DATE_KEY AS DATE_KEY
,DISCOUNT_AMOUNT AS DISCOUNT_AMOUNT
,AMOUNT AS AMOUNT
,QUANTITY_BASE AS QUANTITY_BASE
,QUANTITY AS QUANTITY
,ƯU TIÊN LẤY Ở SALE_INVOICE --> NẾU KHÔNG CÓ Ở SALES_INVOICE MÀ CÓ Ở RETURN THÌ LẤY THEO RETURN AS OFF_ROUTE_FLAG
,ITEM_TYPE AS ITEM_TYPE
,MONTH AS MONTH
,YEAR AS YEAR
,PROMOTION_VALUE_AMOUNT AS PROMOTION_VALUE_AMOUNT
,ISNULL(PRODUCT_KEY,-1) AS PRODUCT_KEY
,ISNULL(SALES_FORCE_KEY,-1) AS SALES_FORCE_KEY
,ISNULL(EMPLOYEE_KEY,-1) AS EMPLOYEE_KEY
,ISNULL(CUSTOMER_KEY,-1) AS CUSTOMER_KEY
,ISNULL(DISTRIBUTOR_KEY,-1) AS DISTRIBUTOR_KEY
,AMOUNT_AFTER_DISCOUNT AS AMOUNT_AFTER_DISCOUNT
,'SALES_INVOICE_DAILY' AS W_SOURCE
,W_I